##### import

In [51]:
import pandas as pd
import numpy as np
import math
import datetime
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
from stock_centre import *
from scipy.stats import norm
import plotly.express as px
hq = Stock_Data_Centre()
pro = TuShare().pro

In [52]:
class Opt_Assistant():
    def __init__(self):
        self.hq = Stock_Data_Centre()
        self.opt_info = pd.read_feather(f"{self.hq.data_path}/opt/contract_info_fix.fea")

    def get_opt_code(self, underlying_symbol, exercise_price, contract_type, exercise_ym=None, date=None):
        '''
        获取期权合约代码
        :param underlying_symbol:  标的代码
        :param exercise_price:  行权价格
        :param contract_type:  合约类型。CO-认购期权，PO-认沽期权
        :param exercise_ym:  到期年月（如:2023-08），与date参数二选一
        :param date:  指定日期，返回全部该日期可交易的标的
        :return:
        '''
        temp_info = self.opt_info.query(
            "underlying_symbol==@underlying_symbol and exercise_price==@exercise_price and contract_type==@contract_type")
        if date is not None:
            date = pd.to_datetime(date)
            temp_info = temp_info.query("list_date<=@date<=last_trade_date")
        elif exercise_ym is not None:
            temp_info = temp_info[temp_info['exercise_ym'] == exercise_ym]
        if len(temp_info) > 0:
            return temp_info.code.tolist()
        return []

    def get_gear_info(self, underlying_symbol, contract_type, exercise_ym, ascending=True, date=None):
        '''
        获取期权档位信息
        :param underlying_symbol: 标的代码
        :param contract_type:  合约类型。CO-认购期权，PO-认沽期权
        :param exercise_ym:  到期年月（如:2023-08）
        :param ascending:  是否升序
        :param date:  指定日期，返回全部该日期可交易的标的，不指定则返回全部。建议输入回测日期，否则可能会返回未上市或已退市的合约
        :return:
        '''
        temp_info = self.opt_info.query(
            "underlying_symbol==@underlying_symbol and contract_type==@contract_type and exercise_ym==@exercise_ym")
        if date is not None:
            date = pd.to_datetime(date)
            temp_info = temp_info.query("list_date<=@date<=last_trade_date")
        temp_info = temp_info[['code', 'name', 'underlying_symbol', 'exchange_code', 'contract_type', 'exercise_price', 'expire_date']].reset_index(drop=True).sort_values(['exercise_price'], ascending=ascending)
        return temp_info
    
    def get_opt_info(self, code):
        '''
        已知合约代码，获取其他详细信息
        :param opt_code:  合约代码
        :param date:  指定日期
        :return:
        '''
        temp_info = self.opt_info.query("code==@code")
        temp_info = temp_info[['code', 'name',  'underlying_symbol', 'exchange_code', 'contract_unit', 'contract_type', 'exercise_price', 'expire_date']].reset_index(drop=True)
        return temp_info

opt_info = Opt_Assistant()

##### set global param

In [65]:
underlying_symbol = '510050'
is_index = False
contract_type = 'CO'
bail_adjust = 0.12
min_multiple = 0.5
start_date = '2021-07-01'
end_date = '2023-06-30'

#### 一、回测前函数定义

##### 标准化函数

In [66]:
def param_standard(underlying_symbol):

    global start_date
    global end_date

    #  标的范围
    index_list = ['000300', '000852', '000016']
    etf_list = ['510050', '510300', '510500', '588000', '588080', '159915', '159901', '159919', '159922']

    #  标的不同系统中的代码和历史价格和
    if underlying_symbol in index_list:  # 指数的交易所为上交所
        underlying_symbol_for_stock = underlying_symbol + '.SH'
        underlying_symbol_for_opt = underlying_symbol + '.XSHG'
        underlying_prc_his = hq.get_hq(code=underlying_symbol_for_stock, start_date=start_date, end_date=end_date,
                                       index_data=True)
        handling_fee = 15
        commission = 15 * 3
    elif underlying_symbol in etf_list:  # etf的交易所按开头分类
        if underlying_symbol.startswith('5'):
            underlying_symbol_for_stock = underlying_symbol + '.SH'
            underlying_symbol_for_opt = underlying_symbol + '.XSHG'
        elif underlying_symbol.startswith('1'):
            underlying_symbol_for_stock = underlying_symbol + '.SZ'
            underlying_symbol_for_opt = underlying_symbol + '.XSHE'
        underlying_prc_his_Tushare = pro.fund_daily(ts_code=underlying_symbol_for_stock,
                                                    start_date=pd.to_datetime(start_date, format='%Y-%m-%d').strftime(
                                                        '%Y%m%d'),
                                                    end_date=pd.to_datetime(end_date, format='%Y-%m-%d').strftime(
                                                        '%Y%m%d'))
        underlying_prc_his = underlying_prc_his_Tushare.set_index(
            pd.to_datetime(underlying_prc_his_Tushare['trade_date'], format='%Y%m%d').dt.strftime('%Y-%m-%d'))
        underlying_prc_his.sort_index(inplace=True)
        handling_fee = 1.3 + 0.3
        commission = 5

    per_fee = handling_fee + commission

    return {'underlying_symbol_for_stock': underlying_symbol_for_stock, 
            'underlying_symbol_for_opt': underlying_symbol_for_opt, 'per_fee': per_fee, 
            'underlying_prc_his': underlying_prc_his}

# a = param_standard(underlying_symbol='510050')['underlying_prc_his']
# a[a.index == '2021-07-05']

,ts_code,trade_date,pre_close,open,high,low,close,change,pct_chg,vol,amount
trade_date,,,,,,,,,,,
2021-07-05,510050.SH,20210705,3.423,3.419,3.435,3.393,3.42,-0.003,-0.0876,4262801.81,1455122.932


##### 交易日函数

In [67]:
def get_trade_dates(underlying_symbol, start_date, end_date):
    '''
    获取某标的在指定日期间的交易日序列
    :param underlying_symbol: 标的代码，仅数字，str格式
    :param start_date: 开始日期
    :param end_date: 截止日期
    :return: # list格式，list中形式为：str '2023-06-16'
    '''
    
    ts_prc_his = param_standard(underlying_symbol)['underlying_prc_his']
    ts_prc_his = ts_prc_his[(ts_prc_his['trade_date'] >= pd.to_datetime(start_date).strftime('%Y-%m-%d'))
                            & (ts_prc_his['trade_date'] <= pd.to_datetime(end_date).strftime('%Y-%m-%d'))]
    if start_date <= end_date:
        dates_info = ts_prc_his.index.tolist()
        # list格式，list中形式为：Timestamp('2023-06-16 00:00:00')
        return dates_info
    else:
        print('起始日期晚于结束日期')
        return []

# print(get_trade_dates(underlying_symbol, start_date, end_date))

def get_trade_date_gap(underlying_symbol, start_date, end_date):
    '''
    获取某标的在指定日期间间隔的交易日天数
    :param underlying_symbol: 标的代码，仅数字，str格式
    :param start_date: 开始日期
    :param end_date: 截止日期
    :return: int格式
    '''
    dates_gap = len(get_trade_dates(underlying_symbol, start_date, end_date))
    return dates_gap

# print(get_trade_date_gap(underlying_symbol, start_date, end_date))

def get_trade_date(underlying_symbol, spec_date):
    '''
    获取某标的距离指定日最近的交易日
    :param underlying_symbol: 指定标的
    :param spec_date: 指定日期，形式'%Y-%m-%d'
    :return:
    '''
    trade_date = get_trade_date(underlying_symbol, spec_date, spec_date)
    while len(trade_date) == 0: # 说明该日非交易日
        spec_date = pd.to_datetime(spec_date)
        spec_date  += relativedelta(days=1)
        trade_date = get_trade_date(underlying_symbol, spec_date, spec_date)

    return spec_date

##### 保证金计算函数

In [70]:
def calculate_init_margin(code, date):
    '''
    计算指定合约在某日的开仓保证金水平
    :param code: 拟交易的期权合约代码，形式‘10004405.XSHG'
    :param date: 指定日期, str '2023-06-16'
    :param bail_adjust: 保证金调整系数
    :param min_multiple: 最低保障倍数
    :return:
    '''

    global hq
    global opt_info
    global underlying_symbol
    global bail_adjust
    global min_multiple
    opt_quotation_info = pd.read_feather(f"{hq.data_path}/quotation/opt/1d/{pd.to_datetime(date).strftime('%Y-%m-%d')}.fea")
    temp_opt_prc_info = opt_quotation_info[opt_quotation_info['code']==code]
    temp_opt_info = opt_info.get_opt_info(code)
    ts_prc_info = param_standard(underlying_symbol)['underlying_prc_his']
    temp_ts_prc_info = ts_prc_info[ts_prc_info.index == pd.to_datetime(date).strftime('%Y-%m-%d')]

    contract_type = temp_opt_info['contract_type'].tolist()[0]
    contract_type = (contract_type=='CO') - (contract_type=='PO') # CO=1,PO=-1
    contract_unit = temp_opt_info['contract_unit'].tolist()[0]
    exercise_price = temp_opt_info['exercise_price'].tolist()[0]
    expire_date = temp_opt_info['expire_date'].tolist()[0]

    opt_pre_settle = temp_opt_prc_info['pre_settle'].tolist()[0]
    ts_pre_close = temp_ts_prc_info['pre_close'].tolist()[0]

    # 期权虚值额 = max(行权价-标的前收盘价, 0) * opt_type
    virtual_pre_value = np.max(exercise_price - ts_pre_close, 0) * contract_type

    # 每手看涨期权保证金=合约乘数*(合约前结算价+max(标的前收盘价×合约保证金调整系数-前虚值额， 最低保障系数×标的前收盘价×合约保证金调整系数))
    # 每手看跌期权保证金=合约乘数*(合约前结算价+max(标的前收盘价×合约保证金调整系数-前虚值额，最低保障系数×合约行权价格×合约保证金调整系数）
    init_margin = contract_unit * (opt_pre_settle + bail_adjust * 
                                   max(ts_pre_close - virtual_pre_value,
                                       min_multiple * (ts_pre_close*(contract_type==1)
                                                       +exercise_price*(contract_type==-1))
                                       )
                                   )
    return init_margin

# print(calculate_init_margin('10004405.XSHG', '2023-03-21'))

def calculate_maintainance_margin(code, date):
    '''
    计算指定合约在某日的开仓保证金水平
    :param code: 拟交易的期权合约代码，形式‘10004405.XSHG'
    :param date: 指定日期, str '2023-06-16'
    :param bail_adjust: 保证金调整系数
    :param min_multiple: 最低保障倍数
    :return:
    '''

    global hq
    global opt_info
    global underlying_symbol
    global bail_adjust
    global min_multiple
    opt_quotation_info = pd.read_feather(f"{hq.data_path}/quotation/opt/1d/{pd.to_datetime(date).strftime('%Y-%m-%d')}.fea")
    temp_opt_prc_info = opt_quotation_info[opt_quotation_info['code']==code]
    temp_opt_info = opt_info.get_opt_info(code)
    ts_prc_info = param_standard(underlying_symbol)['underlying_prc_his']
    temp_ts_prc_info = ts_prc_info[ts_prc_info.index == pd.to_datetime(date).strftime('%Y-%m-%d')]

    contract_type = temp_opt_info['contract_type'].tolist()[0]
    contract_type = (contract_type=='CO') - (contract_type=='PO') # CO=1,PO=-1
    contract_unit = temp_opt_info['contract_unit'].tolist()[0]
    exercise_price = temp_opt_info['exercise_price'].tolist()[0]
    expire_date = temp_opt_info['expire_date'].tolist()[0]

    opt_settle = temp_opt_prc_info['settle_price'].tolist()[0]
    ts_close = temp_ts_prc_info['close'].tolist()[0]

    # 期权虚值额 = max(行权价-标的收盘价, 0) * opt_type
    virtual_pre_value = np.max(exercise_price - ts_close, 0) * contract_type

    # 每手看涨期权保证金=合约乘数*(合约结算价+max(标的收盘价×合约保证金调整系数-虚值额， 最低保障系数×标的收盘价×合约保证金调整系数))
    # 每手看跌期权保证金=合约乘数*(合约结算价+max(标的收盘价×合约保证金调整系数-虚值额，最低保障系数×合约行权价格×合约保证金调整系数))
    maintainance_margin = contract_unit * (opt_settle + bail_adjust *
                                   max(ts_close - virtual_pre_value,
                                       min_multiple * (ts_close*(contract_type==1)
                                                       +exercise_price*(contract_type==-1))
                                       )
                                   )
    return maintainance_margin

# print(calculate_maintainance_margin('10004405.XSHG', '2023-03-21'))